## Noise reduction


In [ ]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
import librosa
from visualization.plots import plot_waveform
from preprocessing.filtering import filter_recordings_30
from preprocessing.noise_filters import bandpass
import noisereduce as nr

### Low pass filter response

##### Intuition behind the filter response plot:
1. Lower frequencies gain is 0dB
2. While frequencies below above 10^2 [rad/s] (cut-off frequency) are silenced (negative gain)

In [ ]:
b, a = signal.butter(4, 100, 'low', analog=True)
w, h = signal.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

### Test on 10Hz signal

##### Testing the low-pass filter
1. Create a signal of 10Hz (signal) and 20Hz (noise)
2. Apply filter to retrieve the signal

In [ ]:
t = np.linspace(0, 1, 1000, False)  # 1 second
sig = np.sin(2*np.pi*10*t) + np.sin(2*np.pi*20*t)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(t, sig)
ax1.set_title('10 Hz and 20 Hz sinusoids')
ax1.axis([0, 1, -2, 2])

sos = signal.butter(10, 15, 'hp', fs=1000, output='sos')
filtered = signal.sosfilt(sos, sig)
ax2.plot(t, filtered)
ax2.set_title('After 15 Hz high-pass filter')
ax2.axis([0, 1, -2, 2])
ax2.set_xlabel('Time [seconds]')
plt.tight_layout()
plt.show()

### Test of the bird recordings (band pass filter)

In [ ]:
df = filter_recordings_30()
df

In [ ]:
sample = df.sample(100, random_state=0)
sample

In [ ]:
for i in range (10):
     filename = f"{sample.iloc[i]['id']}.mp3"
     latin_name = f"{sample.iloc[i]['Latin name']}"
     also = f"{sample.iloc[i]['also']}"
     quality = f"{sample.iloc[i]['q']}"
     sig ,sr = librosa.load(f"/media/jacek/E753-A120/recordings_30/{filename}", sr=None)
     print(f"filename: {filename}\n latin name: {latin_name}\n also: {also}\n quality: {quality}\n")
     
     print("Source")
     plot_waveform(sig, sr)
     print("\n")
     
     print("Bandpass")
     sos = signal.butter(10, 50, 'hp', fs=1000, output='sos')
     filtered = bandpass(sig, [500, 20000], sr)
     plot_waveform(filtered, sr)
     print("\n")
     
     print("Stationary noise reduction")
     filtered = nr.reduce_noise(y = sig, sr = sr, n_std_thresh_stationary = 1.5, stationary = True)
     plot_waveform(filtered, sr)
     print("\n")
        
     print("Non-Stationary noise reduction")
     filtered = nr.reduce_noise(y = sig, sr = sr, thresh_n_mult_nonstationary = 2,stationary = False)
     plot_waveform(filtered, sr)
     print("\n")
     
  
